In [47]:
#13460
from collections import deque

def move_until_stop(x, y, dx, dy, board):
    """
    (x, y) 위치의 구슬을 방향(dx, dy)로 이동시킬 때
    벽(#)을 만나거나 구멍(O)에 빠질 때까지 이동한다.
    이동 후 최종 위치(new_x, new_y)와 이동한 거리(dist)를 반환한다.
    """
    dist = 0
    while True:
        nx = x + dx
        ny = y + dy
        # 벽이면 이동 중단
        if board[nx][ny] == '#':
            break
        # 구멍이면 이동하고 끝냄
        if board[nx][ny] == 'O':
            x, y = nx, ny
            dist += 1
            break
        # 빈 칸이면 계속 이동
        x, y = nx, ny
        dist += 1
    return x, y, dist

def solve():
    N, M = map(int, input().split())
    board = [list(input()) for _ in range(N)]

    # 빨간(R), 파란(B), 구멍(O)의 위치 찾기
    rx = ry = bx = by = 0
    for i in range(N):
        for j in range(M):
            if board[i][j] == 'R':
                rx, ry = i, j
            elif board[i][j] == 'B':
                bx, by = i, j

    # BFS를 위한 큐. 요소: (빨간x, 빨간y, 파란x, 파란y, 움직임횟수)
    queue = deque()
    queue.append((rx, ry, bx, by, 0))

    # 방문 체크용: 빨간/파란 구슬 위치로 표시
    visited = set()
    visited.add((rx, ry, bx, by))

    # 상하좌우 이동 벡터
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while queue:
        rx, ry, bx, by, depth = queue.popleft()

        # 10번 이하로 움직여야 함
        if depth >= 10:
            break

        for dx, dy in directions:
            # 빨간 구슬 이동
            nrx, nry, rdist = move_until_stop(rx, ry, dx, dy, board)
            # 파란 구슬 이동
            nbx, nby, bdist = move_until_stop(bx, by, dx, dy, board)

            # 파란 구슬이 구멍에 빠지면 실패 -> 다음 상태 탐색 스킵
            if board[nbx][nby] == 'O':
                continue

            # 빨간 구슬이 구멍에 빠졌다면 성공 -> depth+1 출력 후 종료
            if board[nrx][nry] == 'O':
                print(depth + 1)
                return

            # 둘 다 구멍이 아닌 곳에 있고, 위치가 같다면(겹치면) 처리
            if nrx == nbx and nry == nby:
                # 이동 거리가 더 많은 쪽을 한 칸 뒤로
                if rdist > bdist:
                    nrx -= dx
                    nry -= dy
                else:
                    nbx -= dx
                    nby -= dy

            # 방문하지 않았다면 큐에 추가
            if (nrx, nry, nbx, nby) not in visited:
                visited.add((nrx, nry, nbx, nby))
                queue.append((nrx, nry, nbx, nby, depth + 1))

    # 10번 이내에 해결하지 못하면 -1 출력
    print(-1)

if __name__ == "__main__":
    solve()

ValueError: not enough values to unpack (expected 2, got 0)

In [45]:
import sys
from collections import deque

def move_left(board):
    """
    주어진 보드를 '왼쪽'으로 이동했을 때의 새로운 보드 상태를 반환한다.
    2048 규칙: 한 번의 이동에서 이미 합쳐진 블록은 다시 합쳐지지 않는다.
    """
    N = len(board)
    new_board = [[0] * N for _ in range(N)]
    
    for i in range(N):
        row = [num for num in board[i] if num != 0]  # 0(빈칸) 제거
        merged = []
        skip = False
        
        for j in range(len(row)):
            if skip:
                skip = False
                continue
            
            # 바로 다음 숫자와 합칠 수 있는지 검사
            if j < len(row) - 1 and row[j] == row[j + 1]:
                merged.append(row[j] * 2)
                skip = True  # 바로 다음 숫자는 이미 합쳐졌으므로 건너뛴다
            else:
                merged.append(row[j])
        
        # 합쳐진 결과를 왼쪽 정렬 상태로 new_board에 복사
        for j in range(len(merged)):
            new_board[i][j] = merged[j]
    
    return new_board

def rotate_90(board):
    """
    보드를 시계 방향으로 90도 회전한 결과를 반환한다.
    """
    N = len(board)
    rotated = [[0]*N for _ in range(N)]
    for r in range(N):
        for c in range(N):
            rotated[c][N-1-r] = board[r][c]
    return rotated

def move(board, direction):
    """
    direction: 0=위, 1=아래, 2=왼쪽, 3=오른쪽
    주어진 board를 해당 방향으로 이동시킨 결과를 반환한다.
    - 핵심 이동 로직은 move_left를 이용하고,
    - 상/하/우 이동은 보드를 회전하거나 뒤집어서 처리한 뒤 다시 복원한다.
    """
    # 위(0)
    if direction == 0:
        # '위로 이동' == 보드를 시계 90도 3번(=반시계 90도 1번) 회전 -> 왼쪽 이동 -> 시계 90도 1번 회전
        temp = rotate_90(board)
        temp = rotate_90(temp)
        temp = rotate_90(temp)
        moved = move_left(temp)
        # 다시 원상 복구(시계 90도 1번)
        moved = rotate_90(moved)
        return moved
    
    # 아래(1)
    elif direction == 1:
        # '아래로 이동' == 보드를 시계 90도 1번 회전 -> 왼쪽 이동 -> 시계 90도 3번 회전(=반시계 90도 1번)
        temp = rotate_90(board)
        moved = move_left(temp)
        # 복구(시계 90도 3번)
        moved = rotate_90(moved)
        moved = rotate_90(moved)
        moved = rotate_90(moved)
        return moved
    
    # 왼쪽(2)
    elif direction == 2:
        return move_left(board)
    
    # 오른쪽(3)
    else:
        # '오른쪽 이동' == 보드를 좌우 반전 -> 왼쪽 이동 -> 다시 좌우 반전
        flipped = [row[::-1] for row in board]
        moved = move_left(flipped)
        moved = [row[::-1] for row in moved]
        return moved

def get_max_tile(board):
    """현재 보드에서 가장 큰 숫자를 반환"""
    return max(max(row) for row in board)

def dfs(board, depth, max_depth):
    """
    DFS(백트래킹)으로 최대 max_depth번 이동했을 때 만들 수 있는
    가장 큰 블록 값을 구한다.
    
    - board: 현재 보드 상태
    - depth: 현재까지 이동한 횟수
    - max_depth: 최대 이동 횟수 (문제 조건=5)
    """
    # 현재 보드에서의 최댓값
    current_max = get_max_tile(board)
    if depth == max_depth:
        return current_max
    
    result = current_max
    for d in range(4):
        new_board = move(board, d)
        # 이동 후 보드가 이전 상태와 같다면 의미가 없으므로 가지치기
        if new_board == board:
            continue
        result = max(result, dfs(new_board, depth + 1, max_depth))
    
    return result

def solve():
    input = sys.stdin.readline
    N = int(input())            # N×N 보드 크기
    board = [list(map(int, input().split())) for _ in range(N)]
    
    # 최대 5번 이동
    answer = dfs(board, 0, 5)
    print(answer)

# 이 solve() 함수를 사용해, 아래처럼 콘솔이나 다른 환경에서 직접 입력을 주고 실행하면 됩니다.
#
# 예) 예제 입력 1
#   3
#   2 2 2
#   4 4 4
#   8 8 8
#
# 실행 결과 -> 16

if __name__ == "__main__":
    solve()

ValueError: invalid literal for int() with base 10: ''

In [49]:
#3190
import sys
from collections import deque

def solve():
    input = sys.stdin.readline
    
    N = int(input().strip())  # 보드의 크기 (N x N)
    K = int(input().strip())  # 사과의 개수
    
    # 사과 위치를 저장 (행, 열) 형태
    apples = set()
    for _ in range(K):
        r, c = map(int, input().split())
        apples.add((r, c))
    
    L = int(input().strip())  # 방향 변환 횟수
    turns = []
    for _ in range(L):
        X, C = input().split()
        X = int(X)
        turns.append((X, C))
    
    # 뱀은 맨 위 맨 좌측 (1,1)에 위치, 길이는 1, 처음에는 오른쪽(동쪽)으로 진행
    # 방향을 0=오른쪽, 1=아래, 2=왼쪽, 3=위 순서로 정의하겠습니다.
    direction = 0
    # 뱀의 몸을 나타내는 자료구조: tail부터 head까지 순서대로 저장 (deque)
    snake = deque()
    snake.append((1, 1))  # 초기 위치
    visited = set()
    visited.add((1, 1))   # 뱀이 차지하는 칸 집합
    
    # 시간, 다음에 회전해야 할 turn의 인덱스
    time = 0
    turn_index = 0
    
    # 방향 벡터 (오른쪽, 아래, 왼쪽, 위)
    moves = [(0,1), (1,0), (0,-1), (-1,0)]
    
    while True:
        # 1초가 지난 뒤 머리를 이동
        time += 1
        head_r, head_c = snake[-1]
        dr, dc = moves[direction]
        
        new_r = head_r + dr
        new_c = head_c + dc
        
        # 1) 벽에 부딪혔는지 확인
        if not (1 <= new_r <= N and 1 <= new_c <= N):
            print(time)
            return
        
        # 2) 자기 자신과 부딪혔는지 확인
        if (new_r, new_c) in visited:
            print(time)
            return
        
        # 새로운 머리를 추가
        snake.append((new_r, new_c))
        visited.add((new_r, new_c))
        
        # 3) 사과가 있는지 확인
        if (new_r, new_c) in apples:
            # 사과가 있다면 사과를 없애고 꼬리는 그대로 두어 길이가 늘어남
            apples.remove((new_r, new_c))
        else:
            # 사과가 없으면 꼬리를 제거(길이 유지)
            tail = snake.popleft()
            visited.remove(tail)
        
        # 4) 현재 시점에 방향 전환이 있는지 확인
        if turn_index < L and time == turns[turn_index][0]:
            # turns[turn_index] = (X, C)
            _, rotate_dir = turns[turn_index]
            if rotate_dir == 'L':
                # 왼쪽 회전 -> direction -= 1
                direction = (direction - 1) % 4
            else:  # 'D'
                # 오른쪽 회전 -> direction += 1
                direction = (direction + 1) % 4
            
            turn_index += 1

if __name__ == "__main__":
    solve()

# 예시 입력 1
# 6
# 3
# 3 4
# 2 5
# 5 3
# 3
# 3 D
# 15 L
# 17 D
# 위 입력에 대해 출력 -> 9

# 예시 입력 2
# 10
# 4
# 1 2
# 1 3
# 1 4
# 1 5
# 4
# 8 D
# 10 D
# 11 D
# 13 L
# 위 입력에 대해 출력 -> 21

# 예시 입력 3
# 10
# 5
# 1 5
# 1 3
# 1 2
# 1 6
# 1 7
# 4
# 8 D
# 10 D
# 11 D
# 13 L
# 위 입력에 대해 출력 -> 13

In [53]:
#14499

import sys

def roll_dice(dice, direction):
    """
    dice = [top, north, east, west, south, bottom]
    direction: 1=동, 2=서, 3=북, 4=남
    
    굴린 뒤 변경된 주사위 상태를 반환한다.
    """
    top, north, east, west, south, bottom = dice
    
    if direction == 1:  # 동쪽으로 굴림
        # 동쪽 굴림: top->west, east->top, west->bottom, bottom->east (north, south 그대로)
        return [west, north, top, bottom, south, east]
    elif direction == 2:  # 서쪽
        # 서쪽 굴림: top->east, west->top, east->bottom, bottom->west (north, south 그대로)
        return [east, north, bottom, top, south, west]
    elif direction == 3:  # 북쪽
        # 북쪽 굴림: top->south, north->top, south->bottom, bottom->north (east, west 그대로)
        return [south, top, east, west, bottom, north]
    else:  # direction == 4, 남쪽
        # 남쪽 굴림: top->north, south->top, north->bottom, bottom->south (east, west 그대로)
        return [north, bottom, east, west, top, south]

def solve():
    input = sys.stdin.readline
    
    N, M, x, y, K = map(int, input().split())
    board = [list(map(int, input().split())) for _ in range(N)]
    commands = list(map(int, input().split()))
    
    # 주사위 초기 상태: 문제에서 설명한 번호를 기준으로
    # index: 0=윗면(1), 1=북(2), 2=동(3), 3=서(4), 4=남(5), 5=바닥(6)
    # 처음에는 모든 면에 0이 적혀 있음
    dice = [0, 0, 0, 0, 0, 0]
    
    # 방향에 따른 이동 (동=1, 서=2, 북=3, 남=4)
    moves = {
        1: (0, 1),   # 동
        2: (0, -1),  # 서
        3: (-1, 0),  # 북
        4: (1, 0)    # 남
    }
    
    for cmd in commands:
        dx, dy = moves[cmd]
        nx, ny = x + dx, y + dy
        
        # 지도의 범위를 벗어나면 명령 무시 (출력 안 함)
        if not (0 <= nx < N and 0 <= ny < M):
            continue
        
        # 주사위를 굴림
        dice = roll_dice(dice, cmd)
        
        # 이동한 칸에 쓰여있는 수 확인
        if board[nx][ny] == 0:
            # 칸에 써 있는 수가 0이면, 주사위 바닥 숫자를 칸에 복사
            board[nx][ny] = dice[5]
        else:
            # 칸에 쓰여 있는 수가 0이 아니면, 칸의 수를 주사위 바닥에 복사 & 칸은 0
            dice[5] = board[nx][ny]
            board[nx][ny] = 0
        
        # 위치 갱신
        x, y = nx, ny
        
        # 이동 후 주사위의 윗면 출력
        print(dice[0])

if __name__ == "__main__":
    solve()

# 예제 입력 1
# 4 2 0 0 8
# 0 2
# 3 4
# 5 6
# 7 8
# 4 4 4 1 3 3 3 2
# 출력 -> 0, 0, 3, 0, 0, 8, 6, 3

# 예제 입력 2
# 3 3 1 1 9
# 1 2 3
# 4 0 5
# 6 7 8
# 1 3 2 2 4 4 1 1 3
# 출력 -> 0, 0, 0, 3, 0, 1, 0, 6, 0


ValueError: not enough values to unpack (expected 5, got 0)

In [55]:
#14500

import sys
sys.setrecursionlimit(10**6)

input = sys.stdin.readline

N, M = map(int, input().split())
board = [list(map(int, input().split())) for _ in range(N)]

# 보드 전체에서의 최댓값(가지치기용)
maxVal = 0
for r in range(N):
    for c in range(M):
        if board[r][c] > maxVal:
            maxVal = board[r][c]

# 방문 여부
visited = [[False]*M for _ in range(N)]

# 상하좌우
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

answer = 0

def dfs(r, c, depth, total):
    """
    깊이 4가 될 때까지 상하좌우로 이동하며,
    방문하지 않은 칸을 순회해 합계(total)를 구한다.
    """
    global answer
    
    # 가지치기: 최대로 얻을 수 있는 이론값 < 현재 answer이면 중단
    if total + (4 - depth) * maxVal <= answer:
        return
    
    # 4칸을 모두 선택했으면 최대값 갱신
    if depth == 4:
        answer = max(answer, total)
        return
    
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < N and 0 <= nc < M and not visited[nr][nc]:
            visited[nr][nc] = True
            dfs(nr, nc, depth+1, total + board[nr][nc])
            visited[nr][nc] = False

def check_t_shape(r, c):
    """
    ㅗ 모양(중심 + 상하좌우 3칸)을 직접 계산.
    중심 칸 + 인접한 3칸을 더해서 최대값을 갱신한다.
    4가지 경우:
     - 위 + 좌 + 우
     - 아래 + 좌 + 우
     - 왼 + 위 + 아래
     - 오른 + 위 + 아래
    """
    global answer
    base = board[r][c]
    
    # 주변에 몇 개의 방향이 가능한지 확인
    # 후보가 4개(상,하,좌,우)여도, 실제로는 3곳만 선택해야 "ㅗ" 형태가 됨
    # 다만 문제 조건상 4방향 중 "정확히 3방향"이 유효하면 적용.
    
    # 1) 위 + 좌 + 우
    if r-1 >= 0 and c-1 >= 0 and c+1 < M:
        s = base + board[r-1][c] + board[r][c-1] + board[r][c+1]
        answer = max(answer, s)
    
    # 2) 아래 + 좌 + 우
    if r+1 < N and c-1 >= 0 and c+1 < M:
        s = base + board[r+1][c] + board[r][c-1] + board[r][c+1]
        answer = max(answer, s)
    
    # 3) 왼 + 위 + 아래
    if r-1 >= 0 and r+1 < N and c-1 >= 0:
        s = base + board[r-1][c] + board[r+1][c] + board[r][c-1]
        answer = max(answer, s)
    
    # 4) 오른 + 위 + 아래
    if r-1 >= 0 and r+1 < N and c+1 < M:
        s = base + board[r-1][c] + board[r+1][c] + board[r][c+1]
        answer = max(answer, s)

# 모든 칸을 시작점으로 DFS + 'ㅗ' 모양 검사
for i in range(N):
    for j in range(M):
        visited[i][j] = True
        dfs(i, j, 1, board[i][j])
        visited[i][j] = False
        
        # ㅗ 모양 체크
        check_t_shape(i, j)

print(answer)

ValueError: not enough values to unpack (expected 2, got 0)

In [59]:
#14502

import sys
from itertools import combinations
from collections import deque

input = sys.stdin.readline

def spread_virus(lab, viruses, N, M):
    """
    바이러스가 상하좌우로 퍼지는 과정을 BFS로 시뮬레이션한다.
    lab: 연구소 지도 (리스트의 리스트)
    viruses: 바이러스 위치 목록
    N, M: 지도 크기
    """
    # lab을 직접 변경하면 원본이 훼손될 수 있으므로, 복사본 사용
    temp = [row[:] for row in lab]
    
    queue = deque(viruses)
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    
    while queue:
        r, c = queue.popleft()
        for dr, dc in directions:
            nr, nc = r+dr, c+dc
            if 0 <= nr < N and 0 <= nc < M:
                if temp[nr][nc] == 0:  # 빈 칸이면 감염
                    temp[nr][nc] = 2
                    queue.append((nr, nc))
    
    # 감염 후 안전영역(0) 개수 반환
    safe_count = 0
    for row in temp:
        safe_count += row.count(0)
    return safe_count

def solve():
    N, M = map(int, input().split())
    lab = []
    empty_positions = []  # 빈 칸(벽을 놓을 수 있는 곳)
    viruses = []          # 바이러스 위치
    
    for i in range(N):
        row = list(map(int, input().split()))
        lab.append(row)
        for j in range(M):
            if row[j] == 0:
                empty_positions.append((i, j))
            elif row[j] == 2:
                viruses.append((i, j))
    
    # 3개의 벽을 세울 빈 칸 조합 모두 시도
    answer = 0
    for walls in combinations(empty_positions, 3):
        # 연구소 지도에 벽 3개 세우기
        for r, c in walls:
            lab[r][c] = 1
        
        # 바이러스 퍼뜨리기 -> 안전 영역 계산
        safe_count = spread_virus(lab, viruses, N, M)
        if safe_count > answer:
            answer = safe_count
        
        # 벽 원상 복구
        for r, c in walls:
            lab[r][c] = 0
    
    print(answer)

if __name__ == "__main__":
    solve()

ValueError: not enough values to unpack (expected 2, got 0)

In [61]:
#14503 

import sys

input = sys.stdin.readline

def solve():
    N, M = map(int, input().split())
    r, c, d = map(int, input().split())
    
    # 방 상태 (0=빈 칸, 1=벽)
    room = [list(map(int, input().split())) for _ in range(N)]
    
    # 방문(청소) 여부를 저장할 배열
    visited = [[False]*M for _ in range(N)]
    
    # 방향 벡터: 0=북, 1=동, 2=남, 3=서
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    def turn_left(d):
        """왼쪽으로 90도 회전"""
        return (d + 3) % 4  # 또는 (d - 1) % 4
    
    # 청소한 칸의 개수
    cleaned_count = 0
    
    # 시뮬레이션 시작
    while True:
        # 1) 현재 칸을 청소
        if not visited[r][c]:
            visited[r][c] = True
            cleaned_count += 1
        
        turned = 0
        moved = False
        
        # 2) 왼쪽 방향으로 회전하면서 갈 곳 탐색 (최대 4번)
        for _ in range(4):
            d = turn_left(d)  # 왼쪽으로 회전
            nr = r + directions[d][0]
            nc = c + directions[d][1]
            
            # 왼쪽 칸이 "빈 칸"이고 아직 청소 안 했다면
            if room[nr][nc] == 0 and not visited[nr][nc]:
                # 전진
                r, c = nr, nc
                moved = True
                break
            else:
                turned += 1
        
        if moved:
            # 새 칸으로 이동했으니 다시 1번부터
            continue
        
        # 3) 네 방향 모두 청소되어 있거나 벽인 경우 -> 뒤로 이동 가능?
        # 뒤쪽 방향 = 현재 d에서 +2 (반대 방향)
        back_d = (d + 2) % 4
        br = r + directions[back_d][0]
        bc = c + directions[back_d][1]
        
        # 뒤가 벽이면 작동 멈춤
        if room[br][bc] == 1:
            break
        else:
            # 뒤로 이동 (방향은 유지)
            r, c = br, bc
    
    # 결과 출력
    print(cleaned_count)

if __name__ == "__main__":
    solve()

ValueError: not enough values to unpack (expected 2, got 0)

In [63]:
#15686

import sys
from itertools import combinations

input = sys.stdin.readline

def calculate_chicken_distance(selected_chickens):
    """
    주어진 M개의 치킨집 조합(selected_chickens)에 대한 도시의 치킨 거리를 계산한다.
    """
    total_distance = 0

    for hx, hy in houses:
        # 현재 집에서 가장 가까운 치킨집까지의 거리 계산
        min_distance = float('inf')
        for cx, cy in selected_chickens:
            distance = abs(hx - cx) + abs(hy - cy)
            min_distance = min(min_distance, distance)
        total_distance += min_distance  # 모든 집의 최소 치킨 거리 합산
    
    return total_distance

def solve():
    N, M = map(int, input().split())
    
    global houses, chicken_shops
    houses = []
    chicken_shops = []

    for i in range(N):
        row = list(map(int, input().split()))
        for j in range(N):
            if row[j] == 1:  
                houses.append((i, j))  # 집 좌표 저장
            elif row[j] == 2:  
                chicken_shops.append((i, j))  # 치킨집 좌표 저장

    # 가능한 치킨집 조합 중 최소 도시 치킨 거리 찾기
    min_city_distance = float('inf')

    # 모든 치킨집 중 M개를 선택하는 조합 탐색
    for selected in combinations(chicken_shops, M):
        city_distance = calculate_chicken_distance(selected)
        min_city_distance = min(min_city_distance, city_distance)

    print(min_city_distance)

if __name__ == "__main__":
    solve()

ValueError: not enough values to unpack (expected 2, got 0)